In [1]:
import mesa

# Data visualization tools.
import seaborn as sns

# Has multi-dimensional arrays and matrices. Has a large collection of
# mathematical functions to operate on these arrays.
import numpy as np

# Data manipulation and analysis.
import pandas as pd

In [29]:
import copy


In [89]:
class CleanAgent(mesa.Agent):
    """Un agente que limpia la celda si está sucia."""

    def __init__(self, unique_id, model):
        super().__init__(unique_id, model)
        self.moves = 0  # Contador de movimientos del agente

    def clean_cell(self):
        """Limpia la celda actual si está sucia."""
        if self.pos in self.model.dirty_cells:
            self.model.dirty_cells.remove(self.pos)

    def move(self):
        """Mueve al agente a una celda vecina aleatoria que esté vacía."""
        possible_steps = self.model.grid.get_neighborhood(
            self.pos, moore=True, include_center=False
        )
        # Filtrar solo las posiciones que están vacías
        empty_steps = [step for step in possible_steps if self.model.is_cell_available(step)]
        if empty_steps:
            new_position = self.random.choice(empty_steps)
            self.model.grid.move_agent(self, new_position)
            self.moves += 1
            self.model.total_moves += 1  # Incrementa el contador total de movimientos

    def step(self):
        """Define las acciones que realiza el agente en cada paso."""
        if self.pos in self.model.dirty_cells:
            self.clean_cell()
        else:
            self.move()

In [90]:
class CleanModel(mesa.Model):
    """Modelo con un número de agentes limpiadores en una cuadrícula toroidal."""

    def __init__(self, NumberAgents=20, widthM=20, heightN=20, dirtycellPercent=0.4, maxTimeExecution=1000):
        super().__init__()
        self.num_agents = NumberAgents
        self.grid = mesa.space.MultiGrid(widthM, heightN, torus=True)  # Usar MultiGrid con torus=True
        self.schedule = mesa.time.RandomActivation(self)
        self.maxTimeExecution = maxTimeExecution
        self.time = 0  # Contador de tiempo
        self.total_moves = 0  # Contador de movimientos
        self.dirty_cells = set()

        # Inicializar celdas sucias
        for x in range(widthM):
            for y in range(heightN):
                if self.random.random() < dirtycellPercent:
                    self.dirty_cells.add((x, y))

        # Crear agentes y ubicarlos en (1, 1)
        for i in range(self.num_agents):
            agent = CleanAgent(i, self)
            self.schedule.add(agent)
            self.grid.place_agent(agent, (1, 1))  # Todos los agentes empiezan en (1, 1)

    def is_cell_available(self, pos):
        """Verifica si una celda está vacía (sin agentes) excepto para la celda inicial (1, 1)."""
        agents = self.grid.get_cell_list_contents([pos])
        if pos == (1, 1):
            # Permitir múltiples agentes en (1, 1)
            return True
        else:
            return len(agents) == 0

    def step(self):
        """Ejecuta un paso de la simulación."""
        self.schedule.step()
        self.time += 1

        # Verificar si todas las celdas están limpias o si se alcanzó el tiempo máximo
        if not self.dirty_cells or self.time >= self.maxTimeExecution:
            self.running = False  # Detiene la simulación

            # Calcular el porcentaje de celdas limpias
            total_cells = self.grid.width * self.grid.height
            clean_cells = total_cells - len(self.dirty_cells)
            percentage_clean = (clean_cells / total_cells) * 100

            # Imprimir o guardar las estadísticas
            print(f"Tiempo total: {self.time}")
            print(f"Porcentaje de celdas limpias: {percentage_clean:.2f}%")
            print(f"Total de movimientos: {self.total_moves}")


In [91]:
model = CleanModel(NumberAgents=20, widthM=40, heightN=40, dirtycellPercent=0.8, maxTimeExecution=1000)
while model.running:
    model.step()

Tiempo total: 1000
Porcentaje de celdas limpias: 99.50%
Total de movimientos: 18674
